In [17]:
import pandas as pd
import sqlite3

# A helper function I wrote to import the .csv files
# into the sqlite database
from helpers import create_table_from_csv


In [18]:
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

create_table_from_csv(cursor, 'department', ['TEXT', 'TEXT'])
create_table_from_csv(cursor, 'employee', ['INT', 'TEXT'])
create_table_from_csv(cursor, 'female', ['TEXT'])
create_table_from_csv(cursor, 'male', ['TEXT'])
create_table_from_csv(cursor, 'project', ['TEXT'])
create_table_from_csv(cursor, 'salary', ['TEXT', 'INT'])
create_table_from_csv(cursor, 'supervise', ['TEXT', 'TEXT'])
create_table_from_csv(cursor, 'workson', ['TEXT', 'TEXT', 'INT'])

conn.commit()
conn.close()

### Q1: Are there any employees that are "female" and that work in project "computerization" with an effort of "10" hours per week, and that have "jennifer" as a supervisor? If so, list them.



In [71]:
conn = sqlite3.connect('database.db')

df = pd.read_sql_query("""
    SELECT
        employee.EMPLOYEE_NAME, workson.PROJECT, workson.EFFORT, supervise.SUPERVISOR
    FROM
        employee
    INNER JOIN 
        female ON employee.EMPLOYEE_NAME = female.NAME
    INNER JOIN 
        workson ON employee.EMPLOYEE_NAME = workson.NAME
    INNER JOIN 
        supervise ON employee.EMPLOYEE_NAME = supervise.SUBORDINATE
    WHERE 
        workson.EFFORT = 10 
        AND 
        workson.PROJECT = 'computerization'
        AND 
        supervise.SUPERVISOR = 'jennifer';                           
""", conn)

conn.close()
df


,EMPLOYEE_NAME,PROJECT,EFFORT,SUPERVISOR
0,alicia,computerization,10,jennifer


### Q2: Who is the employee who makes over "40000" dollars a year and works for the "research" department?

In [72]:
conn = sqlite3.connect('database.db')

df = pd.read_sql_query("""
    SELECT
        employee.EMPLOYEE_NAME, salary.SALARY, department.DEPARTMENT
    FROM 
        employee
    INNER JOIN
        salary ON employee.EMPLOYEE_NAME = salary.EMPLOYEE_NAME
    INNER JOIN
        department ON employee.EMPLOYEE_NAME = department.EMPLOYEE_NAME
    WHERE
        salary.SALARY > 40000
        AND
        department.DEPARTMENT = 'research';
""", conn)

conn.close()
df

,EMPLOYEE_NAME,SALARY,DEPARTMENT
0,franklin,40001,research


### Q3: Who is the supreme chief of this fictional company (aka the President)?

In [73]:
conn = sqlite3.connect('database.db')

df = pd.read_sql_query("""
    SELECT
        employee.EMPLOYEE_NAME, COUNT(supervise.SUPERVISOR)
    FROM 
        employee
    FULL OUTER JOIN
        supervise ON employee.EMPLOYEE_NAME = supervise.SUBORDINATE     
    GROUP BY
        employee.EMPLOYEE_NAME
    HAVING
        COUNT(supervise.SUPERVISOR) = 0;
""", conn)

conn.close()
df

,EMPLOYEE_NAME,COUNT(supervise.SUPERVISOR)
0,james,0


### Q4: Who are the individuals that work on project "productx" with an *effort* of 20 or more hours a week?

In [74]:
conn = sqlite3.connect('database.db')

df = pd.read_sql_query("""
    SELECT
        employee.EMPLOYEE_NAME, workson.PROJECT, workson.EFFORT
    FROM
        employee
    INNER JOIN
        workson ON employee.EMPLOYEE_NAME = workson.NAME
    WHERE
        workson.PROJECT = 'productx'
        AND
        EFFORT >= 20;
""", conn)

conn.close()
df

,EMPLOYEE_NAME,PROJECT,EFFORT
0,john,productx,32
1,joyce,productx,20
